In [10]:
# .npy to nifti converter
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom

def normalize_image(image):
    image_min = image.min()
    image_max = image.max()
    return (image - image_min) / (image_max - image_min + 1e-8)

def resize_and_save_heatmap_to_nifti(npy_path, nifti_path, target_shape=(64, 64, 64)):
    heatmap_data = np.load(npy_path)  # Expected shape: (64, 8, 8)
    print(f"Loaded heatmap shape: {heatmap_data.shape}")

    if heatmap_data.shape[1:] != (8, 8):
        raise ValueError(f"need shape (8, 8), but got {heatmap_data.shape[1:]}")

    resized_volume = []
    for slice_idx in range(heatmap_data.shape[0]):
        heatmap_slice = heatmap_data[slice_idx, :, :]  
        heatmap_slice_norm = normalize_image(heatmap_slice)
        # make viewer happy
        zoom_factors = (
            target_shape[1] / heatmap_slice.shape[0],  # Height scaling
            target_shape[2] / heatmap_slice.shape[1],  # Width scaling
        )
        resized_slice = zoom(heatmap_slice_norm, zoom_factors, order=1)  # Bilinear interpolation
        resized_volume.append(resized_slice)

    resized_volume = np.stack(resized_volume, axis=0)  # Shape: (64, 64, 64)
    print(f"Resized heatmap volume shape: {resized_volume.shape}")

    nifti_img = nib.Nifti1Image(resized_volume, affine=np.eye(4))

    nib.save(nifti_img, nifti_path)
    print(f"NIfTI file saved at: {nifti_path}")

npy_input_path = "/jet/home/abradsha/MEDSYN/results/img_64_standard_bulk/left_pneu_sample_0.npy_token_3_effusion_heatmaps.npy"
nifti_output_path = "/jet/home/abradsha/MEDSYN/results/img_64_standard_bulk/left_pneu_sample_0_effusion_heatmaps.nii.gz"

resize_and_save_heatmap_to_nifti(npy_input_path, nifti_output_path)


Loaded heatmap data with shape: (64, 8, 8)
Resized heatmap volume shape: (64, 64, 64)
NIfTI file saved at: /jet/home/abradsha/MEDSYN/results/img_64_standard_bulk/left_pneu_sample_0_effusion_heatmaps.nii.gz


In [11]:
import numpy as np
import SimpleITK as sitk
from scipy.ndimage import zoom
import os

def normalize_image(image):
    image_min = image.min()
    image_max = image.max()
    return (image - image_min) / (image_max - 1e-8)  # Avoid division by zero

def resize_and_save_heatmap_to_nifti(npy_path, output_dir, output_postfix, target_shape=(64, 64, 64)):
    os.makedirs(output_dir, exist_ok=True)

    heatmap_data = np.load(npy_path)  # Expected shape: (64, 8, 8)
    print(f"Loaded heatmap shape: {heatmap_data.shape}")

    if heatmap_data.shape[1:] != (8, 8):
        raise ValueError(f"Expected heatmap slices to have shape (8, 8), but got {heatmap_data.shape[1:]}")

    resized_volume = []
    for slice_idx in range(heatmap_data.shape[0]):
        heatmap_slice = heatmap_data[slice_idx, :, :]  # Extract individual slice
        heatmap_slice_norm = normalize_image(heatmap_slice)  # Normalize slice

        # Bilinear interpolation to resize
        zoom_factors = (
            target_shape[1] / heatmap_slice.shape[0],  # Height scaling
            target_shape[2] / heatmap_slice.shape[1],  # Width scaling
        )
        resized_slice = zoom(heatmap_slice_norm, zoom_factors, order=1)  # Bilinear interpolation
        resized_volume.append(resized_slice)

    resized_volume = np.stack(resized_volume, axis=0)  # Shape: (64, 64, 64)
    print(f"Resized heatmap volume shape: {resized_volume.shape}")
    nifti_img = sitk.GetImageFromArray(resized_volume)
    output_path = os.path.join(output_dir, f"{output_postfix}.nii.gz")
    sitk.WriteImage(nifti_img, output_path)
    print(f"NIfTI file saved at: {output_path}")
npy_input_path = "/jet/home/abradsha/MEDSYN/results/img_64_standard_bulk/left_pneu_sample_0.npy_token_1_left_heatmaps.npy"
output_dir = "/jet/home/abradsha/MEDSYN/results/img_64_standard_bulk/"
output_postfix = "left_pneu_sample_1_left_heatmaps"

resize_and_save_heatmap_to_nifti(npy_input_path, output_dir, output_postfix)


Loaded heatmap shape: (64, 8, 8)
Resized heatmap volume shape: (64, 64, 64)
NIfTI file saved at: /jet/home/abradsha/MEDSYN/results/img_64_standard_bulk/left_pneu_sample_1_left_heatmaps.nii.gz


In [2]:
#nifit to dicom

import numpy as np
import pydicom
from pydicom.uid import generate_uid
from datetime import datetime
import SimpleITK as sitk
import os

def save_heatmaps_to_dicom(heatmap_volume, output_path, patient_name="HeatmapPatient", 
                           patient_id="HM001", series_description="Heatmaps", modality="OT"):
    if len(heatmap_volume.shape) != 3:
        raise ValueError(f"Expected heatmap volume to have 3 dimensions, but got {heatmap_volume.shape}")

    # dummy data
    ds = pydicom.Dataset()
    ds.file_meta = pydicom.Dataset()
    ds.file_meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian
    ds.is_little_endian = True
    ds.is_implicit_VR = False
    ds.PatientName = patient_name
    ds.PatientID = patient_id
    ds.Modality = modality
    ds.SeriesDescription = series_description
    ds.SeriesInstanceUID = generate_uid()
    ds.StudyInstanceUID = generate_uid()
    ds.SOPInstanceUID = generate_uid()
    ds.StudyDate = datetime.now().strftime("%Y%m%d")
    ds.StudyTime = datetime.now().strftime("%H%M%S")
    ds.ContentDate = ds.StudyDate
    ds.ContentTime = ds.StudyTime
    ds.SamplesPerPixel = 1
    ds.PhotometricInterpretation = "MONOCHROME2"
    ds.Rows, ds.Columns = heatmap_volume.shape[1], heatmap_volume.shape[2]
    ds.NumberOfFrames = heatmap_volume.shape[0]
    ds.PixelRepresentation = 0  # Unsigned integer
    ds.BitsAllocated = 16
    ds.BitsStored = 16
    ds.HighBit = 15

    # Scale heatmap data to fit within 16-bit unsigned integer range
    heatmap_volume = (heatmap_volume * 65535).astype(np.uint16)

    # Convert the heatmap volume to a byte array and store it in PixelData
    ds.PixelData = heatmap_volume.tobytes()
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    pydicom.dcmwrite(output_path, ds)

    print(f"DICOM file saved at: {output_path}")
nifti_path = "/jet/home/abradsha/MEDSYN/results/img_64_standard_bulk/left_pneu_sample_1_left_heatmaps.nii.gz"
heatmap_volume = sitk.GetArrayFromImage(sitk.ReadImage(nifti_path))  # Assuming the file exists and is a 64x64x64 
output_dicom_path = "/jet/home/abradsha/MEDSYN/results/img_64_standard_bulk/heatmap_volume.dcm"
save_heatmaps_to_dicom(heatmap_volume, output_dicom_path)


DICOM file saved at: /jet/home/abradsha/MEDSYN/results/img_64_standard_bulk/heatmap_volume.dcm
